In [ ]:
import astropy.io.fits as fits
from astropy.cosmology import Planck15
import astropy.units as u
import astropy.constants as c
from astropy.visualization import simple_norm
from astropy.modeling import models
from astropy.convolution import convolve

import os
import time
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
plt.rcParams['xtick.labelsize'] = 14.0
plt.rcParams['ytick.labelsize'] = 14.0
import numpy as np

import photutils
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry

import scipy.ndimage as ndi
import statmorph
from statmorph.utils.image_diagnostics import make_figure

In [ ]:
path = os.getcwd()

In [ ]:
def open_file(file_name, path):
    '''Extracts data and header info from a fits file
    
    Parameters:
    file_name <str>: name of file to open with extension
    path <str>: directory were the file is located
    
    Returns:
    data <ndarray>: data from the fits file
    hdr: header with information on the data
    '''
    file = path + file_name
    hdul = fits.open(file)
    hdr = hdul[0].header
    data = hdul[0].data
    hdul.close()
    return data, hdr

# 1. Visualizing the galaxy

## Extracting the data and creating plots (1,2)

In [ ]:
data, hdr = open_file('/galaxy_hydro.fits', path)

In [ ]:
# extracting data for each of the components of the galaxy
stellar_mass = data[0]
form_rate = data[1]
gas_mass = data[2]
dust_mass = data[3]

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(stellar_mass, cmap='magma', 
           norm=simple_norm(stellar_mass, stretch='log'))
plt.contour(dust_mass, cmap='Greens', 
            norm=simple_norm(dust_mass, stretch='log'), linewidths=2)
plt.title('Stellar Mass and Dust Mass Countour', fontsize=20)
plt.xlim(40, 150)
plt.ylim(160, 25)
#plt.savefig('contour1.pdf')
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(form_rate, cmap='magma', 
           norm=simple_norm(form_rate, stretch='log'))
plt.contour(gas_mass, cmap='Greens', 
            norm=simple_norm(stellar_mass, stretch='log'), linewidths=2)
plt.xlim(40, 150)
plt.ylim(160, 25)
plt.title('Star Formation Rate and Gas Mass Contour', fontsize=20)
#plt.savefig('contour2.pdf')
plt.show()

## Calculating total stellar, dust, gas masses and star formation rate (3)

In [ ]:
stellar_t = np.sum(stellar_mass) * u.M_sun
dust_t = np.sum(dust_mass) * u.M_sun
gas_t = np.sum(gas_mass) * u.M_sun
rate_t = np.sum(form_rate) * u.M_sun * u.yr**(-1)

In [ ]:
print('Total Stellar Mass:')
stellar_t

In [ ]:
print('Total Dust Mass:')
dust_t

In [ ]:
print('Total Gas Mass:')
gas_t

In [ ]:
print('Star Formation Rate of Galaxy:')
rate_t

## Plotting the galaxy at different wavelengths (1)

In [ ]:
data_wav, hdr_wav = open_file('/galaxy_allwav.fits', path)
data_wav = data_wav[0]

In [ ]:
# extracting wavelengths from the header
wavelengths = np.array([hdr_wav['IWAV{}'.format(i)] for i in range(20)]) * u.micron

In [ ]:
# picking a few wavelengths to plot
ultraviolet = data_wav[0] 
red = data_wav[1]   
blue = data_wav[3]  
infrared = data_wav[11] 
far_infrared = data_wav[15] 
microwave = data_wav[17]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(14, 10), sharex=True, sharey=True) 

ax[0,0].imshow(ultraviolet, cmap='magma')
ax[0,0].set_title('Ultraviolet', fontsize=18) 

ax[0,1].imshow(blue, cmap='magma')
ax[0,1].set_title('Blue', fontsize=18) 

ax[0,2].imshow(red, cmap='magma')
ax[0,2].set_title('Red', fontsize=18) 

ax[1,0].imshow(infrared, cmap='magma')
ax[1,0].set_title('Infrared', fontsize=18) 

ax[1,1].imshow(far_infrared, cmap='magma')
ax[1,1].set_title('Far-Infrared', fontsize=18)

ax[1,2].imshow(microwave, cmap='magma')
ax[1,2].set_title('Microwave', fontsize=18) 

plt.tight_layout() 
#plt.savefig('wavs.pdf')
plt.show()

In [ ]:
# plotting the far_infrared vs the ultraviolet to show difference in flux measured

fig, ax = plt.subplots(1, 2, figsize=(12, 8), sharey=True) 

ax[0].imshow(ultraviolet, cmap='magma', 
             norm=simple_norm(ultraviolet, stretch='linear'))
ax[0].set_title('Ultraviolet', fontsize=16) 

ax[1].imshow(far_infrared, cmap='magma', 
             norm=simple_norm(ultraviolet, stretch='linear'))
ax[1].set_title('Far-Infrared', fontsize=16) 

plt.tight_layout() 
#plt.savefig('compare.pdf')
plt.show()

## Finding the flux as a function of wavelength (2)

In [ ]:
fluxes = np.zeros(data_wav.shape[0])
for i in range(data_wav.shape[0]):
    dat = data_wav[i]
    fluxes[i] = np.sum(dat)

In [ ]:
gamma = (-100, (1e-12*u.m).to(u.micron).value)
xray = ((1*u.pm).to(u.micron).value, (1*u.nm).to(u.micron).value)
uv = ((1*u.nm).to(u.micron).value, (400*u.nm).to(u.micron).value)
vis = ((400*u.nm).to(u.micron).value, (750*u.nm).to(u.micron).value)
near = ((750*u.nm).to(u.micron).value, (2.5*u.micron).value)
inf = ((2.5*u.micron).value, (25*u.micron).value)
micro = ((25*u.micron).value, (1*u.mm).to(u.micron).value)
radio = ((1*u.mm).to(u.micron).value, 2000)

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(wavelengths, fluxes, 'p', color='black')

"""plt.axvspan(uv[0], uv[1], alpha=0.2, label='ultraviolet', color='royalblue')
plt.axvspan(vis[0], vis[1], alpha=0.2, label='optical', color='green')
plt.axvspan(near[0], near[1], alpha=0.2, label='near-infrared', color='yellow')
plt.axvspan(inf[0], inf[1], alpha=0.2, label='infrared', color='red')
plt.axvspan(micro[0], micro[1], alpha=0.2, label='far_infrared', color='purple')
plt.axvspan(radio[0], radio[1], alpha=0.2, label='microwaves', color='pink')"""

plt.grid(alpha=0.2)
plt.xlabel(r'Wavelength [$\mu$m]', fontsize=20)
plt.ylabel(r'Flux [arb]', fontsize=20)
plt.yscale('log')
plt.xscale('log')
plt.title('Flux of the Galaxy as a Function of Wavelength', fontsize=20)
#plt.legend()
plt.xlim(0.1, 1700)
#plt.savefig('flux.pdf')
plt.show()

# 2. Galaxy Size

## Half-Mass Size

In [ ]:
# setting up variables for the aperture photometry of the total mass of the galaxy
all_masses = stellar_mass + dust_mass + gas_mass
radii = np.linspace(1, 100, 1000)
position = (96., 104.)   # approximately the center of the galaxy

In [ ]:
# performing aperture photometry for r=10
aperture = CircularAperture(position, r=10)
phot_table = aperture_photometry(all_masses, aperture)
phot_table['aperture_sum'].info.format = '%.8g'
print(phot_table)

In [ ]:
def half_size(dat, position, plot=True):
    '''Finds the half size of the galaxy provided the approximate position of its center
    
    Parameters:
    dat <ndarray>: contains the galaxy data
    position <tuple>: approximate coordinates of the center of the galaxy
    plot <bool>: Determines if a plot is created or not
    
    Returns:
    radii[ind] <float>: half-size measurement in pixels
    '''
    diff = []
    half = np.sum(dat)/2
    radii = np.linspace(1, 100, 1000)
    for i, rad in enumerate(radii):
        aperture = CircularAperture(position, r=rad)
        phot_table = aperture_photometry(dat, aperture)
        value = phot_table['aperture_sum'][0]
        d = value - half
        diff.append(np.abs(d))
    ind = np.where(diff == min(diff))
                
    if plot:
        fig, ax = plt.subplots(figsize=(10,10))
        plt.imshow(dat, cmap='magma', norm=simple_norm(dat, stretch='log'))
        circ = plt.Circle((position[1], position[0]), radii[ind], fill=False, linestyle='--', color='white', linewidth=2)
        ax.add_artist(circ)
    
    return (radii[ind])[0]

In [ ]:
def conversions(h_size, ratio=False):
    '''Converts from pixel units to kpc given two scale factors
    
    Parameters:
    h_size <float>: half-size measurement in pixels
    ratio <bool>: defaults to False. When True, estimates pixel scale of the hydro data 
                  file based on ratio of dimensions to allwav file
    Returns:
    result <astropy quantity>: half-size value in units of kpc
    '''
    z = hdr['Z']  # extracting redshift from header
    scale = (Planck15.kpc_proper_per_arcmin(z)).to(u.kpc/u.arcsec)  # using Planck15 package to find scale at z=2
    if ratio:
        arc_scale = (data_wav[0].shape[0]/data[0].shape[0])*(hdr_wav['PIXSCALE']*u.arcsec*u.pix**(-1))
    else:
        arc_scale = hdr_wav['PIXSCALE']*u.arcsec*u.pix**(-1)
        
    result = h_size * u.pix * arc_scale * scale
    return result

In [ ]:
h_size = half_size(all_masses, position, plot=False)
h_size

In [ ]:
half_mass_size = conversions(h_size, True)
half_mass_size

In [ ]:
diff = []
half = np.sum(all_masses)/2
radii = np.linspace(1, 100, 1000)
for i, rad in enumerate(radii):
    aperture = CircularAperture(position, r=rad)
    phot_table = aperture_photometry(all_masses, aperture)
    value = phot_table['aperture_sum'][0]
    d = value - half
    diff.append(np.abs(d))
ind = np.where(diff == min(diff))[0][0]

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(radii, diff, color='black')
plt.plot(radii[ind], diff[ind], 'p', markersize=8, color='black', 
         label='aperture radius = {}'.format(round(radii[ind], 2)))
plt.legend(loc='lower right', fontsize=14)
plt.grid(alpha=0.2)
plt.xlabel('Aperture Radii', fontsize=20)
plt.ylabel('|Half-Mass - Mass in Aperture|', fontsize=20)
plt.title('Aperture Radius for Half-Mass Size', fontsize=20)
plt.show()

## Half-Light Size using Optical Light (1)

In [ ]:
# finding half-light size using optical light of ~500nm
pos = (210, 192)  # approximately center of galaxy
h_size = half_size(blue, pos, plot=True)
half_light = conversions(h_size, False)

In [ ]:
print('The half-light size of the galaxy measure in optical light is:', half_light)
print('The half-mass size of the galaxy is:', half_mass_size)

## Measuring half-light sizes in all wavelengths (2,3)

In [ ]:
half_light_sizes = np.empty(wavelengths.shape)
for i in range(half_light_sizes.shape[0]):
    dat = data_wav[i]
    h_size = half_size(dat, pos, plot=False)
    half_light = conversions(h_size, False)
    half_light_sizes[i] = half_light.value

In [ ]:
line = np.linspace(0.1, 1500, 2)
plt.figure(figsize=(10,7))
plt.plot(wavelengths, half_light_sizes, 'p', color='black')
plt.plot(line, np.ones_like(line)*half_mass_size, '--', 
         color='crimson', label='Half-Mass Size')
plt.xlabel(r'Wavelength [$\mu$m]', fontsize=20)
plt.ylabel('Half-Light Size [kpc]', fontsize=20)
plt.title('Comparing Half-Light Sizes', fontsize=20)
plt.grid(alpha=0.2)
plt.xscale('log')
plt.legend(fontsize=16)
plt.ylim(3,14)
#plt.savefig('sizes.pdf')
plt.show()

## Image of Galaxy on the Sky (4)

In [ ]:
data_sky, hdr_sky = open_file('/galaxy_onsky_F160W.fits', path)

In [ ]:
plt.figure(figsize=(10,7))
plt.title('Sky Data of Galaxy', fontsize=20)
plt.imshow(data_sky, origin='lower')
plt.axis('off')
#plt.savefig('sky.pdf')
plt.show()

# 3. Using statmorph

## statmorph tutorial

## Using statmorph to measure morphological parameters of example galaxy

In [ ]:
threshold = photutils.detect_threshold(data_sky, 3)
npixels = 10  # minimum number of connected pixels
segm = photutils.detect_sources(data_sky, threshold, npixels)

In [ ]:
label = np.argmax(segm.areas) + 1
segmap = segm.data == label
plt.imshow(segmap, origin='lower', cmap='gray')
plt.show()

In [ ]:
segmap_float = ndi.uniform_filter(np.float64(segmap), size=10)
segmap = segmap_float > 0.5
plt.figure(figsize=(10,7))
plt.imshow(segmap, origin='lower', cmap='gray')
plt.axis('off')
plt.title('Segmentation Map for the Main Source', fontsize=20)
#plt.savefig('seg.pdf')
plt.show()

In [ ]:
gain=1.5 # from https://www.stsci.edu/itt/APT_help20/WFC3/c05_detector5.html
source_morphs = statmorph.source_morphology(
    data_sky, segmap, gain=gain)

In [ ]:
morph = source_morphs[0]

In [ ]:
print('xc_centroid =', morph.xc_centroid)
print('yc_centroid =', morph.yc_centroid)
print('ellipticity_centroid =', morph.ellipticity_centroid)
print('elongation_centroid =', morph.elongation_centroid)
print('orientation_centroid =', morph.orientation_centroid)
print('xc_asymmetry =', morph.xc_asymmetry)
print('yc_asymmetry =', morph.yc_asymmetry)
print('ellipticity_asymmetry =', morph.ellipticity_asymmetry)
print('elongation_asymmetry =', morph.elongation_asymmetry)
print('orientation_asymmetry =', morph.orientation_asymmetry)
print('rpetro_circ =', morph.rpetro_circ)
print('rpetro_ellip =', morph.rpetro_ellip)
print('rhalf_circ =', morph.rhalf_circ)
print('rhalf_ellip =', morph.rhalf_ellip)
print('r20 =', morph.r20)
print('r80 =', morph.r80)
print('Gini =', morph.gini)
print('M20 =', morph.m20)
print('F(G, M20) =', morph.gini_m20_bulge)
print('S(G, M20) =', morph.gini_m20_merger)
print('sn_per_pixel =', morph.sn_per_pixel)
print('C =', morph.concentration)
print('A =', morph.asymmetry)
print('S =', morph.smoothness)
print('sersic_amplitude =', morph.sersic_amplitude)
print('sersic_rhalf =', morph.sersic_rhalf)
print('sersic_n =', morph.sersic_n)
print('sersic_xc =', morph.sersic_xc)
print('sersic_yc =', morph.sersic_yc)
print('sersic_ellip =', morph.sersic_ellip)
print('sersic_theta =', morph.sersic_theta)
print('sky_mean =', morph.sky_mean)
print('sky_median =', morph.sky_median)
print('sky_sigma =', morph.sky_sigma)
print('flag =', morph.flag)
print('flag_sersic =', morph.flag_sersic)

In [ ]:
fig = make_figure(morph)

In [ ]:
plt.close(fig)